In [47]:
from colmap_testing.colmap_helpers import read_write_model
import os
import cv2
import numpy as np

In [48]:
path_to_images = os.path.join("colmap_testing", "colmap_ws_aegis_room","images")
image_names = os.listdir(path_to_images)
image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])


In [49]:
path_to_model = os.path.join("colmap_testing", "colmap_ws_aegis_room", "dense","0","sparse")
#path_to_images = os.path.join("colmap_testing", "colmap_ws_aegis_room", "dense", "0", "images")
#image_names = os.listdir(path_to_images)
#image_names = sorted([int(os.path.splitext(name)[0]) for name in image_names])
image_paths = [os.path.join(path_to_images, str(name) + ".png") for name in image_names]

rgb_images = np.array([np.array(cv2.imread(imagepath)) for imagepath in image_paths])
colmap_model = read_write_model.read_model(path_to_model, ".bin")

#cameras:fx, fy, cx, cy, width, height
#images:camera_id, qvec, tvec, name 
#points:xyz, rgb, error, image_ids, point2D_idxs
cameras, images, points = read_write_model.read_model(path_to_model, ".bin")


# 3D point list with one line of data per point:
# POINT3D_ID, X, Y, Z, R, G, B, ERROR, TRACK[] as (IMAGE_ID, POINT2D_IDX)
points_keys = list(points.keys())
random_key = np.random.choice(points_keys)
random_point = points[random_key]

# Image list with two lines of data per image:
#   IMAGE_ID, QW, QX, QY, QZ, TX, TY, TZ, CAMERA_ID, NAME
#   POINTS2D[] as (X, Y, POINT3D_ID)

first_image = images[1]

# Camera list with one line of data per camera:
#   CAMERA_ID, MODEL, WIDTH, HEIGHT, PARAMS[]
first_camera = cameras[1]



#The coordinates of the projection/camera center are given by -R^t * T, 
# where R^t is the inverse/transpose of the 3x3 rotation matrix composed from the quaternion and T is the translation vector. 
# The local camera coordinate system of an image is defined in a way that the X axis points to the right, 
# the Y axis to the bottom, and the Z axis to the front as seen from the image.

#calculate the projection/camera center

In [50]:

#sort keys by name in the dictionary images
def sort_keys_by_name(images):
    keys = images.keys()

    #get each name from the images
    names = []
    for key in keys:
        names.append(images[key].name)
    #remove the extension from the name
    names = [int(os.path.splitext(name)[0]) for name in names]
    sorted_idx = np.argsort(names)
    keys = np.array(list(keys))[sorted_idx]
    return keys

In [51]:
def get_frames(images, cameras, scale=1):
    frames = []
    keys = sort_keys_by_name(images)
    
    for key in keys:
        img = images[key]
        print(img.name)
        # rotation
        R = read_write_model.qvec2rotmat(img.qvec)

        # translation
        t = img.tvec

        # invert
        t = -R.T @ t
        R = R.T

        # intrinsics
        cam = cameras[img.camera_id]
        if cam.model in ("SIMPLE_PINHOLE", "SIMPLE_RADIAL", "RADIAL"):
            fx = fy = cam.params[0]
            cx = cam.params[1]
            cy = cam.params[2]
        elif cam.model in ("PINHOLE", "OPENCV", "OPENCV_FISHEYE", "FULL_OPENCV"):
            fx = cam.params[0]
            fy = cam.params[1]
            cx = cam.params[2]
            cy = cam.params[3]
        else:
            raise Exception("Camera model not supported")

        # intrinsics
        K = np.identity(3)
        K[0, 0] = fx
        K[1, 1] = fy
        K[0, 2] = cx
        K[1, 2] = cy
        frames.append((R, t, K))
    return frames

#frames are composed of R, t, K
frames = get_frames(images, cameras)

random_point_3d = random_point[1] #3d point coordinates xyz
print(random_point_3d)

9.png
19.png
28.png
37.png
46.png
55.png
64.png
121.png
144.png
167.png
211.png
234.png
257.png
301.png
324.png
347.png
392.png
414.png
437.png
482.png
504.png
527.png
572.png
595.png
[5.65072284 4.04810207 8.21696423]


In [52]:
R, t, K = frames[0]
#add nested to t
t = np.array([t]).T
print(R)
print(t)
print(np.hstack((R, t)))

[[ 0.98901725  0.00372603  0.14775314]
 [ 0.00664032  0.99755256 -0.06960462]
 [-0.14765087  0.0698213   0.98657195]]
[[ 4.65185843]
 [ 3.23646824]
 [-0.75078046]]
[[ 9.89017252e-01  3.72602605e-03  1.47753143e-01  4.65185843e+00]
 [ 6.64032371e-03  9.97552556e-01 -6.96046211e-02  3.23646824e+00]
 [-1.47650874e-01  6.98212998e-02  9.86571946e-01 -7.50780457e-01]]


In [54]:
#project 3d point using the camera projection matrix onto the rgb image
#project them all into the same image
from matplotlib import pyplot as plt

subplots = []
for i,frame in enumerate(frames):
    R, t, K = frame
    #add nested to t
    t = np.array([t]).T
    P = K @ np.hstack((R, t)) #projection matrix
    point_on_image = P @ np.hstack((random_point_3d, 1))
    point_on_image = point_on_image / point_on_image[2]
    print(point_on_image)
    plt.scatter(point_on_image[0], point_on_image[1], c="r")
    plt.imshow(rgb_images[i])
    plt.savefig(os.path.join("colmap_testing","reprojections","colmap_test_reproj{}.png".format(i)))
    #plt.show()
    plt.close()
    

    



[1.85335080e+03 1.30830039e+03 1.00000000e+00]
[1.70776699e+03 1.17146550e+03 1.00000000e+00]
[1.65368725e+03 1.08552494e+03 1.00000000e+00]
[1.63912660e+03 1.06562528e+03 1.00000000e+00]
[1.58436153e+03 1.06760245e+03 1.00000000e+00]
[1.54139047e+03 1.05907551e+03 1.00000000e+00]
[1.37965962e+03 1.02128322e+03 1.00000000e+00]
[456.95009155 872.3667805    1.        ]
[862.80188308 873.29629993   1.        ]
[1.22960618e+03 9.58917295e+02 1.00000000e+00]
[1.03753542e+03 8.78679413e+02 1.00000000e+00]
[861.76377414 831.68077335   1.        ]
[695.94642226 779.52202654   1.        ]
[675.25318955 829.01724072   1.        ]
[482.21391309 862.21560713   1.        ]
[406.49257574 904.2651811    1.        ]
[442.22136456 962.74800047   1.        ]
[6.09295574e+02 1.10607194e+03 1.00000000e+00]
[6.60395142e+02 1.11963973e+03 1.00000000e+00]
[1.35405175e+03 6.37984428e+02 1.00000000e+00]
[1.70052677e+03 9.89021992e+02 1.00000000e+00]
[1.35444544e+03 1.12122875e+03 1.00000000e+00]
[1.1205269e+03